# 📊 ВИЗУАЛИЗАЦИЯ ОЧИСТКИ ДАННЫХ

Визуализация процесса очистки и подготовки данных


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Настройка стиля графиков
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 10

# Папка для сохранения
viz_dir = 'png/cleaning'
os.makedirs(viz_dir, exist_ok=True)

%matplotlib inline


In [ ]:
# Загрузка данных
print("📥 Загрузка данных...")
df_balanced = pd.read_csv('dataset_balanced.csv')
print(f"✅ Загружено {len(df_balanced):,} записей")

try:
    df_original = pd.read_csv('dataset.csv')
    print(f"✅ Загружено {len(df_original):,} записей из исходного датасета")
    has_original = True
except:
    print("⚠️ Исходный датасет не найден")
    has_original = False
    df_original = None

df = df_balanced.copy()
df['text_length'] = df['text'].str.len()
df['word_count'] = df['text'].str.split().str.len()


In [ ]:
# Визуализация процесса очистки данных
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('📊 ВИЗУАЛИЗАЦИЯ ОЧИСТКИ ДАННЫХ', fontsize=16, fontweight='bold')

# 1. Сравнение размеров датасетов
if has_original:
    ax1 = axes[0, 0]
    sizes = [len(df_original), len(df_balanced)]
    labels = ['Исходный\nдатасет', 'После\nочистки']
    colors = ['#ff6b6b', '#51cf66']
    bars = ax1.bar(labels, sizes, color=colors, alpha=0.8, edgecolor='black', linewidth=2)
    ax1.set_ylabel('Количество записей', fontsize=12)
    ax1.set_title('Размер датасета до и после очистки', fontsize=13, fontweight='bold')
    ax1.grid(axis='y', alpha=0.3)
    
    for bar, size in zip(bars, sizes):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{size:,}\n({size/len(df_original)*100:.1f}%)',
                ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    reduction = (1 - len(df_balanced)/len(df_original)) * 100
    ax1.text(0.5, max(sizes) * 0.5, f'Удалено: {reduction:.1f}%',
            ha='center', fontsize=12, style='italic', 
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
else:
    axes[0, 0].text(0.5, 0.5, 'Исходный датасет\nне найден', 
                   ha='center', va='center', fontsize=14)
    axes[0, 0].set_title('Размер датасета', fontsize=13)

# 2. Распределение длины текстов
ax2 = axes[0, 1]
df['text_length'].hist(bins=50, ax=ax2, color='#4dabf7', alpha=0.7, edgecolor='black')
ax2.axvline(df['text_length'].mean(), color='red', linestyle='--', linewidth=2, label=f'Среднее: {df["text_length"].mean():.1f}')
ax2.axvline(df['text_length'].median(), color='green', linestyle='--', linewidth=2, label=f'Медиана: {df["text_length"].median():.1f}')
ax2.set_xlabel('Длина текста (символы)', fontsize=11)
ax2.set_ylabel('Частота', fontsize=11)
ax2.set_title('Распределение длины текстов', fontsize=13, fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

# 3. Распределение количества слов
ax3 = axes[1, 0]
df['word_count'].hist(bins=50, ax=ax3, color='#ff8787', alpha=0.7, edgecolor='black')
ax3.axvline(df['word_count'].mean(), color='red', linestyle='--', linewidth=2, label=f'Среднее: {df["word_count"].mean():.1f}')
ax3.set_xlabel('Количество слов', fontsize=11)
ax3.set_ylabel('Частота', fontsize=11)
ax3.set_title('Распределение количества слов', fontsize=13, fontweight='bold')
ax3.legend()
ax3.grid(alpha=0.3)

# 4. Boxplot длины текстов по категориям
ax4 = axes[1, 1]
category_order = df['category'].value_counts().index
df_sorted = df.sort_values('category', key=lambda x: x.map({cat: i for i, cat in enumerate(category_order)}))
box_plot = df_sorted.boxplot(column='text_length', by='category', ax=ax4, 
                             patch_artist=True, grid=False)
ax4.set_title('Длина текстов по категориям', fontsize=13, fontweight='bold')
ax4.set_xlabel('Категория', fontsize=11)
ax4.set_ylabel('Длина текста (символы)', fontsize=11)
plt.setp(ax4.xaxis.get_majorticklabels(), rotation=45, ha='right')
ax4.grid(alpha=0.3)

plt.tight_layout()
save_path = os.path.join(viz_dir, 'cleaning_overview.png')
plt.savefig(save_path, dpi=150, bbox_inches='tight')
print(f"✅ График сохранен: {save_path}")
plt.show()
